In [11]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [12]:
rr = RandomRules(
  TOTAL_NUMBERS = 50,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [45, 46, 47, 48, 49, 50]
)

In [13]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=41'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [43]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_b5 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_b2)
  cols.append(post_str_b3)
  cols.append(post_str_b4)
  cols.append(post_str_b5)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_b2 = str_combo[1]
  post_str_b3 = str_combo[2]
  post_str_b4 = str_combo[3]
  post_str_b5 = str_combo[4]
  post_str_blast = str_combo[-1]

with open('./gn_tnk_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas', 'yapa', 'adicionales', 'sorteo_extra',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=2475


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""11/05/2025""","""1193""","""46 21 49 45 03 43""","""17""","""20 30 32 44""","""Promoción Sí o Sí""","""032143454649""","[3, 21, … 49]",3,21,43,45,46,49,"""03""","""21""","""43""","""45""","""46""","""49""",null,null,null,null,null,null,false,0,"""18_22_02_01_03""",-59
"""07/05/2025""","""1192""","""48 02 38 50 33 35""","""49""","""20 16""","""Promoción Sí o Sí""","""023335384850""","[2, 33, … 50]",2,33,35,38,48,50,"""02""","""33""","""35""","""38""","""48""","""50""","""03""","""21""","""43""","""45""","""46""","""49""",true,16,"""31_02_03_10_02""",3
"""04/05/2025""","""1191""","""09 35 19 15 37 10""","""46""","""25 02 38""","""Promoción Sí o Sí""","""091015193537""","[9, 10, … 37]",9,10,15,19,35,37,"""09""","""10""","""15""","""19""","""35""","""37""","""02""","""33""","""35""","""38""","""48""","""50""",true,16,"""01_05_04_16_02""",-57
"""30/04/2025""","""1190""","""11 36 33 06 02 16""","""37""","""14 07 10 01""","""Promoción Sí o Sí""","""020611163336""","[2, 6, … 36]",2,6,11,16,33,36,"""02""","""06""","""11""","""16""","""33""","""36""","""09""","""10""","""15""","""19""","""35""","""37""",true,8,"""04_05_05_17_03""",-51
"""27/04/2025""","""1189""","""46 33 21 08 07 41""","""39""","""36 09 37""","""Promoción Sí o Sí""","""070821334146""","[7, 8, … 46]",7,8,21,33,41,46,"""07""","""08""","""21""","""33""","""41""","""46""","""02""","""06""","""11""","""16""","""33""","""36""",false,0,"""01_13_12_08_05""",-6
"""23/04/2025""","""1188""","""22 16 20 33 35 09""","""46""","""24""","""Promoción Sí o Sí""","""091620223335""","[9, 16, … 35]",9,16,20,22,33,35,"""09""","""16""","""20""","""22""","""33""","""35""","""07""","""08""","""21""","""33""","""41""","""46""",false,0,"""07_04_02_11_02""",-39
"""20/04/2025""","""1187""","""33 02 06 42 34 11""","""37""","""49 20""","""Promoción Sí o Sí""","""020611333442""","[2, 6, … 42]",2,6,11,33,34,42,"""02""","""06""","""11""","""33""","""34""","""42""","""09""","""16""","""20""","""22""","""33""","""35""",false,0,"""04_05_22_01_08""",120
"""16/04/2025""","""1186""","""14 31 46 38 48 12""","""15""","""39 30""","""Promoción Sí o Sí""","""121431384648""","[12, 14, … 48]",12,14,31,38,46,48,"""12""","""14""","""31""","""38""","""46""","""48""","""02""","""06""","""11""","""33""","""34""","""42""",false,0,"""02_17_07_08_02""",-54
"""13/04/2025""","""1185""","""22 36 39 48 46 44""","""28""","""09 40""","""Promoción Sí o Sí""","""223639444648""","[22, 36, … 48]",22,36,39,44,46,48,"""22""","""36""","""39""","""44""","""46""","""48""","""12""","""14""","""31""","""38""","""46""","""48""",true,24,"""14_03_05_02_02""",26


In [44]:
dfRows.item(0, 'b2')

21

In [45]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )

  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [46]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='03'
df_viz FOR str_b1 = 03


post_str_b1,count,percent
str,u32,f64
"""02""",31,13.191489
"""03""",28,11.914894
"""01""",25,10.638298
"""04""",21,8.93617
"""05""",14,5.957447
…,…,…
"""17""",2,0.851064
"""25""",2,0.851064
"""28""",2,0.851064


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 03
Last 3 results last_results=['05', '01', '18']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""23/03/2025""","""1179""","""24 07 18 03 37 49""","""43""","""50 28""","""Promoción Sí o Sí""","""030718243749""","[3, 7, … 49]",3,7,18,24,37,49,"""03""","""07""","""18""","""24""","""37""","""49""","""01""","""05""","""13""","""35""","""37""","""43""",false,0,"""04_11_06_13_12""",-44
"""09/02/2025""","""1167""","""34 03 46 35 33 12""","""18""","""44 06 41""","""Promoción Sí o Sí""","""031233343546""","[3, 12, … 46]",3,12,33,34,35,46,"""03""","""12""","""33""","""34""","""35""","""46""","""18""","""23""","""24""","""32""","""33""","""44""",true,8,"""09_21_01_01_11""",-62
"""20/11/2024""","""1144""","""36 27 28 26 29 03""","""10""","""44 39 45""","""Promoción Sí o Sí""","""032627282936""","[3, 26, … 36]",3,26,27,28,29,36,"""03""","""26""","""27""","""28""","""29""","""36""","""05""","""19""","""26""","""29""","""45""","""46""",true,24,"""23_01_01_01_07""",28
"""28/08/2024""","""1120""","""11 37 46 47 06 03""","""29""","""14 09 05 40""","""Promoción Sí o Sí""","""030611374647""","[3, 6, … 47]",3,6,11,37,46,47,"""03""","""06""","""11""","""37""","""46""","""47""","""04""","""19""","""23""","""37""","""41""","""50""",true,17,"""03_05_26_09_01""",104
"""28/07/2024""","""1111""","""19 14 25 03 08 26""","""50""","""""","""""","""030814192526""","[3, 8, … 26]",3,8,14,19,25,26,"""03""","""08""","""14""","""19""","""25""","""26""","""07""","""12""","""22""","""28""","""29""","""37""",false,0,"""05_06_05_06_01""",-12
"""14/07/2024""","""1107""","""43 03 34 05 42 26""","""29""","""08 33""","""Promoción Sí o Sí""","""030526344243""","[3, 5, … 43]",3,5,26,34,42,43,"""03""","""05""","""26""","""34""","""42""","""43""","""01""","""19""","""38""","""40""","""43""","""47""",true,24,"""02_21_08_08_01""",-65
"""12/06/2024""","""1098""","""03 11 48 08 32 16""","""09""","""23""","""Promoción Sí o Sí""","""030811163248""","[3, 8, … 48]",3,8,11,16,32,48,"""03""","""08""","""11""","""16""","""32""","""48""","""02""","""08""","""19""","""21""","""23""","""46""",true,8,"""05_03_05_16_16""",-25
"""09/06/2024""","""1097""","""39 20 29 03 07 11""","""27""","""21 08""","""Promoción Sí o Sí""","""030711202939""","[3, 7, … 39]",3,7,11,20,29,39,"""03""","""07""","""11""","""20""","""29""","""39""","""03""","""08""","""11""","""16""","""32""","""48""",true,9,"""04_04_09_09_10""",16
"""14/02/2024""","""1064""","""42 03 18 07 29 39""","""30""","""28 31""","""Promoción Sí o Sí""","""030718293942""","[3, 7, … 42]",3,7,18,29,39,42,"""03""","""07""","""18""","""29""","""39""","""42""","""12""","""23""","""24""","""31""","""35""","""42""",true,8,"""04_11_11_10_03""",-11


df_jumps_map FOR str_b1 = 03


jumps_map,count
str,u32
"""01_07_19_05_03""",1
"""03_07_11_03_05""",1
"""13_02_07_07_13""",1
"""09_05_05_02_21""",1
"""05_03_05_16_16""",1
…,…
"""28_03_04_05_01""",1
"""08_02_07_03_04""",1
"""10_01_03_19_02""",1


df_unijump FOR str_b1 = 03


unijump,count
i64,u32
-25,6
4,6
-23,5
34,5
-33,5
…,…
-92,1
52,1
-83,1


In [47]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='21'
df_viz FOR str_b2 = 21


post_str_b2,count,percent
str,u32,f64
"""10""",7,11.47541
"""08""",5,8.196721
"""16""",4,6.557377
"""15""",4,6.557377
"""07""",4,6.557377
…,…,…
"""13""",1,1.639344
"""09""",1,1.639344
"""23""",1,1.639344


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 21
Last 3 results last_results=['20', '15', '17']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""18/09/2024""","""1126""","""26 10 34 37 21 32""","""45""","""20 15""","""Promoción Sí o Sí""","""102126323437""","[10, 21, … 37]",10,21,26,32,34,37,"""10""","""21""","""26""","""32""","""34""","""37""","""04""","""17""","""20""","""37""","""38""","""48""",false,0,"""11_05_06_02_03""",22
"""22/05/2024""","""1092""","""21 31 47 15 39 33""","""01""","""28 43""","""Promoción Sí o Sí""","""152131333947""","[15, 21, … 47]",15,21,31,33,39,47,"""15""","""21""","""31""","""33""","""39""","""47""","""18""","""20""","""25""","""27""","""29""","""39""",true,2,"""06_10_02_06_08""",-38
"""17/04/2024""","""1082""","""47 40 26 21 06 27""","""34""","""35 11""","""Promoción Sí o Sí""","""062126274047""","[6, 21, … 47]",6,21,26,27,40,47,"""06""","""21""","""26""","""27""","""40""","""47""","""02""","""15""","""16""","""29""","""34""","""36""",true,16,"""15_05_01_13_07""",-44
"""24/03/2024""","""1075""","""32 21 09 48 25 28""","""39""","""22""","""Promoción Sí o Sí""","""092125283248""","[9, 21, … 48]",9,21,25,28,32,48,"""09""","""21""","""25""","""28""","""32""","""48""","""06""","""11""","""16""","""19""","""23""","""30""",false,0,"""12_04_03_04_16""",14
"""01/11/2023""","""1034""","""35 23 21 46 11 28""","""25""","""03 32""","""Promoción Sí o Sí""","""112123283546""","[11, 21, … 46]",11,21,23,28,35,46,"""11""","""21""","""23""","""28""","""35""","""46""","""03""","""14""","""19""","""25""","""32""","""34""",false,0,"""10_02_05_07_11""",15
"""25/10/2023""","""1032""","""38 17 22 26 21 36""","""07""","""27 29 11""","""Promoción Sí o Sí""","""172122263638""","[17, 21, … 38]",17,21,22,26,36,38,"""17""","""21""","""22""","""26""","""36""","""38""","""06""","""10""","""11""","""19""","""36""","""39""",false,0,"""04_01_04_10_02""",-14
"""30/07/2023""","""1007""","""40 27 18 44 21 39""","""30""","""13 48""","""Promoción Sí o Sí""","""182127394044""","[18, 21, … 44]",18,21,27,39,40,44,"""18""","""21""","""27""","""39""","""40""","""44""","""05""","""12""","""16""","""33""","""38""","""42""",false,0,"""03_06_12_01_04""",51
"""17/05/2023""","""0986""","""43 38 15 21 33 28""","""08""","""40 13""","""Promoción Sí o Sí""","""152128333843""","[15, 21, … 43]",15,21,28,33,38,43,"""15""","""21""","""28""","""33""","""38""","""43""","""05""","""12""","""18""","""29""","""35""","""45""",true,8,"""06_07_05_05_05""",-7
"""22/01/2023""","""0953""","""39 25 32 21 24 08""","""42""","""37 41""","""Promoción Sí o Sí""","""082124253239""","[8, 21, … 39]",8,21,24,25,32,39,"""08""","""21""","""24""","""25""","""32""","""39""","""08""","""11""","""17""","""24""","""29""","""38""",true,8,"""13_03_01_07_07""",-14


df_jumps_map FOR str_b2 = 21


jumps_map,count
str,u32
"""16_05_05_06_03""",1
"""03_01_14_06_01""",1
"""03_09_06_05_04""",1
"""16_04_07_08_05""",1
"""19_07_10_02_02""",1
…,…
"""17_01_16_01_06""",1
"""09_03_09_03_05""",1
"""11_04_03_02_03""",1


df_unijump FOR str_b2 = 21


unijump,count
i64,u32
3,3
-14,2
-17,2
15,2
5,2
…,…
22,1
-10,1
46,1


In [48]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='43'


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [49]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='45'


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [50]:
possible_5, exclude_jump_maps_5, last_post_5 = plot_post_col(col='str_b6', post_col='post_str_5')

Setting test_value='49'


ColumnNotFoundError: unable to find column "post_str_5"; valid columns: ["fecha", "sorteo", "bolillas", "yapa", "adicionales", "sorteo_extra", "id", "combo", "b1", "b2", "b3", "b4", "b5", "b6", "str_b1", "str_b2", "str_b3", "str_b4", "str_b5", "str_b6", "post_str_b1", "post_str_b2", "post_str_b3", "post_str_b4", "post_str_b5", "post_str_blast", "is_rare_combination", "reason", "jumps_map", "unijump"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["fecha", "sorteo", "bolillas", "yapa", ...]; PROJECT */30 COLUMNS

In [51]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b6', post_col='post_str_blast')

Setting test_value='49'
df_viz FOR str_b6 = 49


post_str_blast,count,percent
str,u32,f64
"""47""",3,30.0
"""42""",1,10.0
"""49""",1,10.0
"""48""",1,10.0
"""43""",1,10.0
"""35""",1,10.0
"""46""",1,10.0
"""39""",1,10.0


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b6 = 49
Last 3 results last_results=['43', '48', '49']


fecha,sorteo,bolillas,yapa,adicionales,sorteo_extra,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""09/04/2025""","""1184""","""35 24 49 01 12 39""","""21""","""13 07 26""","""Promoción Sí o Sí""","""011224353949""","[1, 12, … 49]",1,12,24,35,39,49,"""01""","""12""","""24""","""35""","""39""","""49""","""22""","""36""","""39""","""44""","""46""","""48""",false,0,"""11_12_11_04_10""",23
"""06/04/2025""","""1183""","""34 42 05 49 16 27""","""29""","""37""","""Promoción Sí o Sí""","""051627344249""","[5, 16, … 49]",5,16,27,34,42,49,"""05""","""16""","""27""","""34""","""42""","""49""","""01""","""12""","""24""","""35""","""39""","""49""",true,8,"""11_11_07_08_07""",-16
"""23/03/2025""","""1179""","""24 07 18 03 37 49""","""43""","""50 28""","""Promoción Sí o Sí""","""030718243749""","[3, 7, … 49]",3,7,18,24,37,49,"""03""","""07""","""18""","""24""","""37""","""49""","""01""","""05""","""13""","""35""","""37""","""43""",false,0,"""04_11_06_13_12""",-44
"""16/03/2025""","""1177""","""15 30 49 12 05 27""","""23""","""24""","""Promoción Sí o Sí""","""051215273049""","[5, 12, … 49]",5,12,15,27,30,49,"""05""","""12""","""15""","""27""","""30""","""49""","""04""","""32""","""34""","""37""","""41""","""42""",false,0,"""07_03_12_03_19""",74
"""22/01/2025""","""1162""","""21 10 49 31 32 06""","""14""","""39 29 36""","""Promoción Sí o Sí""","""061021313249""","[6, 10, … 49]",6,10,21,31,32,49,"""06""","""10""","""21""","""31""","""32""","""49""","""13""","""18""","""26""","""30""","""33""","""35""",false,0,"""04_11_10_01_17""",33
"""12/01/2025""","""1159""","""49 32 41 44 26 34""","""14""","""43""","""Promoción Sí o Sí""","""263234414449""","[26, 32, … 49]",26,32,34,41,44,49,"""26""","""32""","""34""","""41""","""44""","""49""","""16""","""19""","""36""","""38""","""46""","""47""",false,0,"""06_02_07_03_05""",33
"""05/01/2025""","""1157""","""18 49 45 13 35 22""","""21""","""19 07 36 08""","""Promoción Sí o Sí""","""131822354549""","[13, 18, … 49]",13,18,22,35,45,49,"""13""","""18""","""22""","""35""","""45""","""49""","""20""","""24""","""25""","""40""","""44""","""47""",false,0,"""05_04_13_10_04""",31
"""22/12/2024""","""1153""","""17 02 49 40 24 01""","""45""","""14 22 05""","""Promoción Sí o Sí""","""010217244049""","[1, 2, … 49]",1,2,17,24,40,49,"""01""","""02""","""17""","""24""","""40""","""49""","""02""","""05""","""14""","""15""","""24""","""39""",true,16,"""01_15_07_16_09""",-72
"""13/11/2024""","""1142""","""49 19 35 09 17 18""","""45""","""34""","""Promoción Sí o Sí""","""091718193549""","[9, 17, … 49]",9,17,18,19,35,49,"""09""","""17""","""18""","""19""","""35""","""49""","""07""","""11""","""15""","""37""","""42""","""46""",true,24,"""08_01_01_16_14""",-40


df_jumps_map FOR str_b6 = 49


jumps_map,count
str,u32
"""04_11_10_01_17""",1
"""11_11_07_08_07""",1
"""01_15_07_16_09""",1
"""06_02_07_03_05""",1
"""04_11_06_13_12""",1
"""05_04_13_10_04""",1
"""11_12_11_04_10""",1
"""07_03_12_03_19""",1
"""13_13_09_06_07""",1


df_unijump FOR str_b6 = 49


unijump,count
i64,u32
33,2
-72,1
-16,1
-40,1
31,1
23,1
74,1
-44,1
-2,1


In [52]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [53]:
parquet_file = './parquet_tnk_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []
  post_str_b1 = None
  post_str_blast = None
  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_blast = str_combo[-1]

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
      'post_str_b1', 'post_str_blast',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 6),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

parquet_file='./parquet_tnk_v3.parquet' does not exists. creating
TOTAL_LINES=15890700 loaded to parquet_file='./parquet_tnk_v3.parquet'


id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""010203040506""","[1, 2, … 6]",1,2,3,4,5,6,"""01""","""02""","""03""","""04""","""05""","""06""",null,null,true,8,"""01_01_01_01_01""",0
"""010203040507""","[1, 2, … 7]",1,2,3,4,5,7,"""01""","""02""","""03""","""04""","""05""","""07""","""01""","""06""",true,9,"""01_01_01_01_02""",1
"""010203040508""","[1, 2, … 8]",1,2,3,4,5,8,"""01""","""02""","""03""","""04""","""05""","""08""","""01""","""07""",true,8,"""01_01_01_01_03""",2
"""010203040509""","[1, 2, … 9]",1,2,3,4,5,9,"""01""","""02""","""03""","""04""","""05""","""09""","""01""","""08""",true,8,"""01_01_01_01_04""",3
"""010203040510""","[1, 2, … 10]",1,2,3,4,5,10,"""01""","""02""","""03""","""04""","""05""","""10""","""01""","""09""",true,8,"""01_01_01_01_05""",4
"""010203040511""","[1, 2, … 11]",1,2,3,4,5,11,"""01""","""02""","""03""","""04""","""05""","""11""","""01""","""10""",true,9,"""01_01_01_01_06""",5
"""010203040512""","[1, 2, … 12]",1,2,3,4,5,12,"""01""","""02""","""03""","""04""","""05""","""12""","""01""","""11""",true,8,"""01_01_01_01_07""",6
"""010203040513""","[1, 2, … 13]",1,2,3,4,5,13,"""01""","""02""","""03""","""04""","""05""","""13""","""01""","""12""",true,9,"""01_01_01_01_08""",7
"""010203040514""","[1, 2, … 14]",1,2,3,4,5,14,"""01""","""02""","""03""","""04""","""05""","""14""","""01""","""13""",true,8,"""01_01_01_01_09""",8


In [60]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  # pl.col("str_b3").is_in(possible_b3),
  # pl.col("str_b4").is_in(possible_b4),
  # pl.col("str_b5").is_in(possible_b5),
  pl.col("str_b6").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  # ~pl.col("str_b3").is_in(last_post_b3),
  # ~pl.col("str_b4").is_in(last_post_b4),
  # ~pl.col("str_b5").is_in(last_post_b5),

  ~pl.col("str_b6").is_in(last_post_blast),

  # ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_b5 + exclude_jump_maps_blast),
  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""020607103047""","[2, 6, … 47]",2,6,7,10,30,47,"""02""","""06""","""07""","""10""","""30""","""47""","""02""","""46""",false,0,"""04_01_03_20_17""",-45
"""020607103247""","[2, 6, … 47]",2,6,7,10,32,47,"""02""","""06""","""07""","""10""","""32""","""47""","""02""","""46""",false,0,"""04_01_03_22_15""",-55
"""020607103347""","[2, 6, … 47]",2,6,7,10,33,47,"""02""","""06""","""07""","""10""","""33""","""47""","""02""","""46""",false,0,"""04_01_03_23_14""",-60
"""020607103447""","[2, 6, … 47]",2,6,7,10,34,47,"""02""","""06""","""07""","""10""","""34""","""47""","""02""","""46""",false,0,"""04_01_03_24_13""",-65
"""020607103547""","[2, 6, … 47]",2,6,7,10,35,47,"""02""","""06""","""07""","""10""","""35""","""47""","""02""","""46""",false,0,"""04_01_03_25_12""",-70
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""132441444547""","[13, 24, … 47]",13,24,41,44,45,47,"""13""","""24""","""41""","""44""","""45""","""47""","""13""","""46""",false,0,"""11_17_03_01_02""",-41
"""132441444647""","[13, 24, … 47]",13,24,41,44,46,47,"""13""","""24""","""41""","""44""","""46""","""47""","""13""","""50""",false,0,"""11_17_03_02_01""",-46
"""132442434647""","[13, 24, … 47]",13,24,42,43,46,47,"""13""","""24""","""42""","""43""","""46""","""47""","""13""","""50""",false,0,"""11_18_01_03_01""",-66


In [61]:
df_possible.sample(n=1)


id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""021213252847""","[2, 12, … 47]",2,12,13,25,28,47,"""02""","""12""","""13""","""25""","""28""","""47""","""02""","""46""",false,0,"""10_01_12_03_19""",85
